In [ ]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
# downloading the semantic segmentation model
model = torch.hub.load('pytorch/vision:v0.5.0', 'deeplabv3_resnet101', pretrained=True)
model.eval()

In [ ]:
import urllib
"""
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)
"""
filename = '/home/sidroy/Downloads/chair_views/chair5.jpg'
#filename = '/home/sidroy/Insight/projects/Pix2Vox/LargeDatasets/Pix3D/img/sofa/0128.jpg'
segment_image(filename,padding=0,img_number=4)

In [ ]:
img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
scale_percent = 517/img.shape[0]*100 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
print(scale_percent,'%')
dim = (width, height)
print("Dimensions of input to segmentation code {}".format(dim))
# resize image
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
cv2.imwrite('segmentation_image.png', img)
filename = 'segmentation_image.png'

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
# Input image is opened in this line
input_image = Image.open(filename)
print("Shape of Input Image {}".format(input_image.size))
dim_input_image = len(input_image.size)
"""
if dim_input_image == 2:
    input_image = input_image.resize(1,input_image.size[0],input_image.size[1])
"""

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# preprocessed image after all the transformations
input_tensor = preprocess(input_image)
print("Input Tensor Size is {}".format(input_tensor.shape))
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    # output is the pytorch tensor output by the segmentation model
    # it contains the probability of each pixel being ina certain class
    output = model(input_batch)['out'][0]
# output predictions is array of pixels with max probability of being in a class
output_predictions = output.argmax(0)
print('output shape is {}'.format(output.shape))
print('output_predictions is of shape {}'.format(output_predictions.shape))

In [ ]:
# create a color pallette, selecting a color for each class
palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# plot the semantic segmentation predictions of 21 classes in each color
r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize(input_image.size)
r.putpalette(colors)

import matplotlib.pyplot as plt
%matplotlib notebook
plt.imshow(r)
plt.show()

In [ ]:
%matplotlib notebook
plt.imshow(  input_tensor.permute(1, 2, 0)  )

In [ ]:
%matplotlib notebook
mask = output_predictions.cpu()
plt.imshow( mask )

categories, counts = np.unique(mask, return_counts=True)
if len(counts) == 1:
    max_count_index = np.argmax(counts)+1
else:
    max_count_index = np.argmax(counts[1:])+1
    
category = categories[max_count_index]
print(category)
    

In [ ]:
import cv2
res = np.array(input_image)
res[60, 60]
b_channel, g_channel, r_channel = cv2.split(res)
print(b_channel.shape)
print(g_channel.shape)
print(r_channel.shape)

mask_array = np.asarray(mask)
mask_array = np.where(mask!=category, 0, 255)
mask_array = mask_array.astype(np.uint8)
print( type(mask_array) ) 
print( type(b_channel)  )

alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 50
img_BGRA = cv2.merge((b_channel, g_channel, r_channel, mask_array))
img_RGBA = cv2.merge(( r_channel, g_channel, b_channel, mask_array))

image_width = img_RGBA.shape[1]
image_height = img_RGBA.shape[0]
scale_percent = 500/image_width*100
print(scale_percent)
width = int(img_RGBA.shape[1] * scale_percent / 100)
height = int(img_RGBA.shape[0] * scale_percent / 100)
dim = (width, height)
resized_RGBA = cv2.resize(img_RGBA, dim, interpolation = cv2.INTER_AREA)

In [ ]:
%matplotlib notebook
plt.imshow(img_BGRA)
plt.show()
cv2.imwrite('/home/sidroy/Insight/projects/Pix2Vox/datasets/DemoImage/car/car_subfolder/rendering/segmented.png', resized_RGBA) 

In [ ]:
desired_size = 137
extra_padding = 0
im = resized_RGBA
print("Read image size {}".format(im.shape))

old_size = im.shape[:2] # old_size is in (height, width) format
print("Old Size is {}".format(old_size))

ratio = float(desired_size)/max(old_size)
new_size = tuple([int(x*ratio) for x in old_size])

print("New Size is {}".format(new_size))

# new_size should be in (width, height) format

im = cv2.resize(im, (new_size[1], new_size[0]))

delta_w = desired_size - new_size[1]
delta_h = desired_size - new_size[0]
top, bottom = delta_h//2 + extra_padding, delta_h-(delta_h//2) + extra_padding
left, right = delta_w//2 + extra_padding, delta_w-(delta_w//2) + extra_padding

color = [0, 0, 0]
new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
    value=color)
new_im = cv2.resize(new_im, (desired_size,desired_size), interpolation = cv2.INTER_AREA)
print('final size = {}'.format(new_im.shape))
plt.imshow(new_im)

cv2.imwrite('/home/sidroy/Insight/projects/Pix2Vox/datasets/DemoImage/car/car_subfolder/rendering/00.png', new_im)

In [ ]:
import glob
import os

file_list = glob.glob(os.path.join(os.getcwd(), "load_images", "*.*"))

print(file_list)

i = 0
for file_path in file_list:
    img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    img_name = '{:02d}'.format(i)
    cv2.imwrite(img_name+'.png',img)
    i+=1
    

In [ ]:
folder_name = "load_images"
file_list = glob.glob(os.path.join(os.getcwd(), folder_name, "*.*"))

In [ ]:
print(file_list)

In [ ]:
def segment_image(filename,padding,img_number):
    import torch
    import numpy as np
    import cv2
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    scale_percent = 517/img.shape[0]*100 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    print(scale_percent,'%')
    dim = (width, height)
    print("Dimensions of input to segmentation code {}".format(dim))
    # resize image
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite('segmentation_image.png', img)
    filename = 'segmentation_image.png'
    
    # sample execution (requires torchvision)
    from PIL import Image
    from torchvision import transforms
    # Input image is opened in this line
    input_image = Image.open(filename)
    print("Shape of Input Image {}".format(input_image.size))
    dim_input_image = len(input_image.size)
    """
    if dim_input_image == 2:
        input_image = input_image.resize(1,input_image.size[0],input_image.size[1])
    """

    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    # preprocessed image after all the transformations
    input_tensor = preprocess(input_image)
    print("Input Tensor Size is {}".format(input_tensor.shape))
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        # output is the pytorch tensor output by the segmentation model
        # it contains the probability of each pixel being ina certain class
        output = model(input_batch)['out'][0]
    # output predictions is array of pixels with max probability of being in a class
    output_predictions = output.argmax(0)
    print('output shape is {}'.format(output.shape))
    print('output_predictions is of shape {}'.format(output_predictions.shape))

    %matplotlib notebook
    mask = output_predictions.cpu()
    plt.imshow( mask )

    categories, counts = np.unique(mask, return_counts=True)
    if len(counts) == 1:
        max_count_index = np.argmax(counts)+1
    else:
        max_count_index = np.argmax(counts[1:])+1

    category = categories[max_count_index]
    print(category)

    import cv2
    res = np.array(input_image)
    res[60, 60]
    b_channel, g_channel, r_channel = cv2.split(res)
    print(b_channel.shape)
    print(g_channel.shape)
    print(r_channel.shape)

    mask_array = np.asarray(mask)
    mask_array = np.where(mask!=category, 0, 255)
    mask_array = mask_array.astype(np.uint8)
    print( type(mask_array) ) 
    print( type(b_channel)  )

    alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 50
    img_BGRA = cv2.merge((b_channel, g_channel, r_channel, mask_array))
    img_RGBA = cv2.merge(( r_channel, g_channel, b_channel, mask_array))

    image_width = img_RGBA.shape[1]
    image_height = img_RGBA.shape[0]
    scale_percent = 137/image_width*100
    print(scale_percent)
    width = int(img_RGBA.shape[1] * scale_percent / 100)
    height = int(img_RGBA.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_RGBA = cv2.resize(img_RGBA, dim, interpolation = cv2.INTER_AREA)

    desired_size = 137
    extra_padding = padding
    im = resized_RGBA
    print("Read image size {}".format(im.shape))

    old_size = im.shape[:2] # old_size is in (height, width) format
    print("Old Size is {}".format(old_size))

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    print("New Size is {}".format(new_size))

    # new_size should be in (width, height) format

    im = cv2.resize(im, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2 + extra_padding, delta_h-(delta_h//2) + extra_padding
    left, right = delta_w//2 + extra_padding, delta_w-(delta_w//2) + extra_padding

    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
    new_im = cv2.resize(new_im, (desired_size,desired_size), interpolation = cv2.INTER_AREA)
    print('final size = {}'.format(new_im.shape))
    plt.imshow(new_im)

    cv2.imwrite('/home/sidroy/Insight/projects/Pix2Vox/datasets/DemoImage/car/car_subfolder/rendering/{:02d}.png'.format(img_number), new_im)